# Новый препроцессинг

In [1]:
import json
import nltk
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from subprocess import Popen
from collections import Counter

In [2]:
with open('data/corpus.json', 'r', encoding='utf-8') as f:
    corpus = json.load(f)

In [3]:
'Текстов -- {}, слов -- {}.'.format(len(corpus),
                                    sum([len(sample) for sample in corpus]))

'Текстов -- 1157366, слов -- 16028874.'

In [4]:
' '.join(corpus[0])

'оказалось для пользовании этой услугой банку нужно предоставить фото отчет накладных и магазина склада'

In [5]:
sents = [' '.join(sent) for sent in corpus]
batches = np.array_split(sents, 15)
[len(batch) for batch in batches]

[77158,
 77158,
 77158,
 77158,
 77158,
 77158,
 77158,
 77158,
 77158,
 77158,
 77158,
 77157,
 77157,
 77157,
 77157]

Части речи у Майстема бывают [вот такие](https://yandex.ru/dev/mystem/doc/grammemes-values.html), но в нашем оценочном датасете они обозначаются по-другому, поэтому обозначения нужно унифицировать (но не все, а только существительные, прилагательные и глаголы, потому что больше в оценочном датасете ничего нет).

In [6]:
def unify_pos(word_pos):
    if word_pos == 'A':
        return 'ADJ'
    if word_pos == 'S':
        return 'NOUN'
    if word_pos == 'V':
        return 'VERB'
    if word_pos == 'ADV':
        return 'ADV'
    if word_pos == 'PR':
        return 'PREP'
    if 'NUM' in word_pos:
        return 'NUM'
    if 'PRO' in word_pos:
        return 'PRON'
    return word_pos

In [7]:
stopwords = nltk.corpus.stopwords.words('russian')

In [8]:
def parse_line(line, stopwords=stopwords):

    if 'analysis' in line:  # есть только у слов

        if line['analysis']:  # если не распозналось, это мусор
            analysis = line['analysis'][0]
            lem = analysis['lex']  # лемма

            # сразу убираем стоп-слова
            if lem not in stopwords:
                ps = analysis['gr'].split(',')[0].split('=')[0]  # pos
                res = lem + '_' + unify_pos(ps)
                return res

    elif '^' in line['text']:
        res = '^'
        return res

In [9]:
clean_lines = []

for batch in tqdm(batches):

    # создаем исходный файл с разделителем
    joined_batch = '\n^\n'.join(batch)
    with open('raw_batch.txt', 'w', encoding='utf-8') as f:
        f.write(joined_batch)

    # парсим консольным майстемом
    Popen(r'.\mystem.exe -c -n -d -i --format json raw_batch.txt parsed_batch.json',
          shell=True).wait()

    # считываем предобработанный батч
    lines = []
    with open('parsed_batch.json', 'r', encoding='utf-8') as f:
        for line in f.readlines():
            lines.append(json.loads(line))

    # получаем нужную информацию
    for line in tqdm(lines):
        clean_line = parse_line(line)
        if clean_line:
            clean_lines.append(clean_line)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2267053 [00:00<?, ?it/s]

  0%|          | 0/2269919 [00:00<?, ?it/s]

  0%|          | 0/2269583 [00:00<?, ?it/s]

  0%|          | 0/2271909 [00:00<?, ?it/s]

...

  0%|          | 0/2275311 [00:00<?, ?it/s]

  0%|          | 0/2275442 [00:00<?, ?it/s]

  0%|          | 0/2271866 [00:00<?, ?it/s]

  0%|          | 0/2272096 [00:00<?, ?it/s]

  0%|          | 0/2253424 [00:00<?, ?it/s]

In [11]:
clean_corpus = []
one_text = []

for i, line in enumerate(clean_lines):
    if line == '^':
        clean_corpus.append(one_text)
        one_text = []
    elif i == len(clean_lines) - 1:
        clean_corpus.append(one_text)
    else:
        one_text.append(line)

clean_corpus = [t for t in clean_corpus if t]

In [12]:
len(clean_corpus)

1157224

# Соберем частотный словарь

In [13]:
all_lemmas = []
for text in clean_corpus:
    all_lemmas.extend(text)

In [14]:
Counter(all_lemmas).most_common(10)

[('число_NOUN', 419528),
 ('банк_NOUN', 311595),
 ('карта_NOUN', 156216),
 ('кредит_NOUN', 86866),
 ('деньги_NOUN', 78313),
 ('это_PRON', 77045),
 ('день_NOUN', 74886),
 ('сотрудник_NOUN', 69477),
 ('весь_PRON', 66929),
 ('счет_NOUN', 64141)]

In [17]:
freq_df = pd.DataFrame.from_records(Counter(all_lemmas).most_common(),
                                    columns=['word', 'freq'])
freq_df

,word,freq
0,число_NOUN,419528
1,банк_NOUN,311595
2,карта_NOUN,156216
3,кредит_NOUN,86866
4,деньги_NOUN,78313
...,...,...
66603,непогасить_VERB,1
66604,посейдон_NOUN,1
66605,троллят_NOUN,1
66606,забоиться_VERB,1


In [19]:
# уникальных слов в словаре
freq_df.shape[0]

66608

In [20]:
# всего слов в очищенном корпусе
n_words = freq_df.freq.sum()
n_words

10034226

# Замена редких слов
В нашем корпусе осталось много слов, которые встречаются очень редко. Давайте мы редкие слова заменим на специальный токе UNK - unknown. Так мы разительно сократим размер нашего словаря слов с незначительной потерей информации.

In [22]:
print('Доля слов, которые мы заменим на UNK:')

for threshold in np.arange(5, 31, 5):

    sub_df = freq_df[freq_df.freq < threshold]

    unk_freq = sub_df['freq'].sum() * 100 / n_words

    print('Порог отсечения - {}, доля UNK - {:.2f} %, осталось {} слов, удалили {} слов'.format(
        threshold, unk_freq, freq_df.shape[0] - sub_df.shape[0], sub_df.shape[0]))

Доля слов, которые мы заменим на UNK:
Порог отсечения - 5, доля UNK - 0.67 %, осталось 21059 слов, удалили 45549 слов
Порог отсечения - 10, доля UNK - 1.05 %, осталось 15248 слов, удалили 51360 слов
Порог отсечения - 15, доля UNK - 1.37 %, осталось 12567 слов, удалили 54041 слов
Порог отсечения - 20, доля UNK - 1.65 %, осталось 10897 слов, удалили 55711 слов
Порог отсечения - 25, доля UNK - 1.88 %, осталось 9807 слов, удалили 56801 слов
Порог отсечения - 30, доля UNK - 2.10 %, осталось 9002 слов, удалили 57606 слов


Видно, что наш препроцессинг уменьшил количество мусора в корпусе, поэтому сейчас отсеивается меньше слов. Тем не менее слов стало меньше, чем было, поэтому уменьшим и threshold.

In [23]:
threshold = 10
vocab = freq_df[freq_df.freq >= threshold]

In [24]:
words = set(vocab.word)

In [25]:
len(words)

15248

In [27]:
'Мы сократили наш словарь в {:.2f} раз с потерей 1.05 % всех слов.'.format(freq_df.shape[0] / len(words))

'Мы сократили наш словарь в 4.37 раз с потерей 1.05 % всех слов.'

In [28]:
def get_correct_words(word):
    
    if word in words:
        return word
    else:
        return 'UNK'

In [29]:
# заменим слово токеном UNK, если его нет в нашем новом словаре
processed_corpus = [[get_correct_words(tok) for tok in text] for text in tqdm(clean_corpus)]

  0%|          | 0/1157224 [00:00<?, ?it/s]

In [30]:
def drop_duplicate_unks(tokens):

    output_tokens = []

    for tok in tokens:

        if tok == 'UNK' and output_tokens and output_tokens[-1] == 'UNK':
            continue

        output_tokens.append(tok)

    return output_tokens

In [31]:
# дедублируем подряд идущие унки (оставим только один)
processed_corpus = [drop_duplicate_unks(sample) for sample in tqdm(processed_corpus)]

  0%|          | 0/1157224 [00:00<?, ?it/s]

In [32]:
random.shuffle(processed_corpus)

In [36]:
with open('data/new_corpus.json', 'w', encoding='utf-8') as f:
    json.dump(processed_corpus[:100000], f)

In [38]:
with open('data/full_new_corpus.json', 'w', encoding='utf-8') as f:
    json.dump(processed_corpus[:200000], f)